In [ ]:
import torch
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
cd ..


In [ ]:
pwd

In [ ]:
cd drive


In [ ]:
cd My Drive

In [ ]:
cd DL_assignment/

In [ ]:
new_train_data = np.load('new_train_data_v4.npy')
new_train_gt = np.load('new_train_gt_v4.npy')
new_val_data = np.load('new_val_data_v4.npy')
new_val_gt = np.load('new_val_gt_v4.npy')

In [ ]:
import torch
import numpy as np
from torch.utils import data



tensor_x = torch.Tensor(new_train_data) # transform to torch tensor
tensor_y = torch.Tensor(new_train_gt)

my_dataset = data.TensorDataset(tensor_x,tensor_y) # create your datset
my_dataloader = data.DataLoader(my_dataset)


tensor_vx = torch.Tensor(new_val_data)
tensor_vy = torch.Tensor(new_val_gt)

my_val = data.TensorDataset(tensor_vx,tensor_vy) # create your datset
my_valoader = data.DataLoader(my_val)

In [ ]:
from torch import nn
import torch.nn.functional as F

class SegNet(nn.Module):
    
    def __init__(self):
        super(SegNet, self).__init__()
        
        self.conv1_1 = nn.Conv2d(1, 64, kernel_size = 3, stride = 1, padding = 1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1)
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size = 3, stride = 1, padding = 1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size = 3, stride = 1, padding = 1)
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size = 3, stride = 1, padding = 1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1)

        self.pool1 = nn.Conv2d(64, 64, kernel_size = 2, stride = 2, padding = 0)
        self.pool2 = nn.Conv2d(128, 128, kernel_size = 2, stride = 2, padding = 0)
        self.pool3 = nn.Conv2d(256, 256, kernel_size = 2, stride = 2, padding = 0)
        self.pool4 = nn.Conv2d(512, 512, kernel_size = 2, stride = 2, padding = 0)
        self.pool5 = nn.Conv2d(512, 512, kernel_size = 2, stride = 2, padding = 0)
        
        self.unpool5 = nn.ConvTranspose2d(512, 512, kernel_size = 2, stride = 2, padding = 0)
        self.unpool4 = nn.ConvTranspose2d(512, 512, kernel_size = 2, stride = 2, padding = 0)
        self.unpool3 = nn.ConvTranspose2d(256, 256, kernel_size = 2, stride = 2, padding = 0)
        self.unpool2 = nn.ConvTranspose2d(128, 128, kernel_size = 2, stride = 2, padding = 0)
        self.unpool1 = nn.ConvTranspose2d(64, 64,kernel_size = 2, stride = 2, padding = 0)

        self.deconv5_1 = nn.ConvTranspose2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.deconv5_2 = nn.ConvTranspose2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.deconv5_3 = nn.ConvTranspose2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.deconv4_1 = nn.ConvTranspose2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.deconv4_2 = nn.ConvTranspose2d(512, 512, kernel_size = 3, stride = 1, padding = 1)
        self.deconv4_3 = nn.ConvTranspose2d(512, 256, kernel_size = 3, stride = 1, padding = 1)
        self.deconv3_1 = nn.ConvTranspose2d(256, 256, kernel_size = 3, stride = 1, padding = 1)
        self.deconv3_2 = nn.ConvTranspose2d(256, 256, kernel_size = 3, stride = 1, padding = 1)
        self.deconv3_3 = nn.ConvTranspose2d(256, 128, kernel_size = 3, stride = 1, padding = 1)
        self.deconv2_1 = nn.ConvTranspose2d(128, 128, kernel_size = 3, stride = 1, padding = 1)
        self.deconv2_2 = nn.ConvTranspose2d(128, 64, kernel_size = 3, stride = 1, padding = 1)
        self.deconv1_1 = nn.ConvTranspose2d(64, 64, kernel_size = 3, stride = 1, padding = 1)
        self.deconv1_2 = nn.ConvTranspose2d(64, 3, kernel_size = 3, stride = 1, padding = 1)

        self.batch_norm1 = nn.BatchNorm2d(64)
        self.batch_norm2 = nn.BatchNorm2d(128)
        self.batch_norm3 = nn.BatchNorm2d(256)
        self.batch_norm4 = nn.BatchNorm2d(512)

    def forward(self, x):
        
        size_1 = x.size()
        x = self.conv1_1(x)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = self.conv1_2(x)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x= self.pool1(x)
        
        size_2 = x.size()
        x = self.conv2_1(x)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = self.conv2_2(x)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = self.pool2(x)
        
        size_3 = x.size()
        x = self.conv3_1(x)
        x = self.batch_norm3(x)
        x = F.relu(x)
        x = self.conv3_2(x)
        x = self.batch_norm3(x)
        x = F.relu(x)
        x = self.conv3_3(x)
        x = self.batch_norm3(x)
        x = F.relu(x)
        x = self.pool3(x)
        
        size_4 = x.size()
        x = self.conv4_1(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.conv4_2(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.conv4_3(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.pool4(x)

        size_5 = x.size()
        x = self.conv5_1(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.conv5_2(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.conv5_3(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.pool5(x)

        
        x = self.unpool5(x)
        x = self.deconv5_1(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.deconv5_2(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.deconv5_3(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        
        x = self.unpool4(x)
        x = self.deconv4_1(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.deconv4_2(x)
        x = self.batch_norm4(x)
        x = F.relu(x)
        x = self.deconv4_3(x)
        x = self.batch_norm3(x)
        x = F.relu(x)
        
        x = self.unpool3(x)
        x = self.deconv3_1(x)
        x = self.batch_norm3(x)
        x = F.relu(x)
        x = self.deconv3_2(x)
        x = self.batch_norm3(x)
        x = F.relu(x)
        x = self.deconv3_3(x)
        x = self.batch_norm2(x)
        x = F.relu(x)
        
        x = self.unpool2(x)
        x = self.deconv2_1(x)
        x = self.batch_norm2(x)
        x = F.relu(x)
        x = self.deconv2_2(x)
        x = self.batch_norm1(x)
        x = F.relu(x)
        
        x = self.unpool1(x)
        x = self.deconv1_1(x)
        x = self.batch_norm1(x)
        x = F.relu(x)
        x = self.deconv1_2(x)
        output = F.softmax(x, dim = 1)
        return x, output

def dice_loss(true, logits, eps=1e-7):
    """Computes the Sørensen–Dice loss.
    Note that PyTorch optimizers minimize a loss. In this
    case, we would like to maximize the dice loss so we
    return the negated dice loss.
    Args:
        true: a tensor of shape [B, 1, H, W].
        logits: a tensor of shape [B, C, H, W]. Corresponds to
            the raw output or logits of the model.
        eps: added to the denominator for numerical stability.
    Returns:
        dice_loss: the Sørensen–Dice loss.
    """
    num_classes = logits.shape[1]
    if num_classes == 1:
        true_1_hot = torch.eye(num_classes + 1)[true.squeeze(1)]
        true_1_hot = true_1_hot.permute(0, 3, 1, 2).float()
        true_1_hot_f = true_1_hot[:, 0:1, :, :]
        true_1_hot_s = true_1_hot[:, 1:2, :, :]
        true_1_hot = torch.cat([true_1_hot_s, true_1_hot_f], dim=1)
        pos_prob = torch.sigmoid(logits)
        neg_prob = 1 - pos_prob
        probas = torch.cat([pos_prob, neg_prob], dim=1)
    else:
        true_1_hot = torch.eye(num_classes)[true.squeeze(1)]
        true_1_hot = true_1_hot.permute(0, 3, 1, 2).float()
        probas = F.softmax(logits, dim=1)
    true_1_hot = true_1_hot.type(logits.type())
    dims = (0,) + tuple(range(2, true.ndimension()))
    intersection = torch.sum(probas * true_1_hot, dims)
    cardinality = torch.sum(probas + true_1_hot, dims)
    dice_loss = (2. * intersection / (cardinality + eps)).mean()
    # print(dice_loss, 1 - dice_loss)
    return (dice_loss)

def cal_confusion_matrix(output, label, val):
  # if val == False:
  output = output.cpu().detach()
  output = np.array(output)
  label = label.cpu().detach()
  label = np.array(label)
  confu_mat = np.zeros((3, 3))
  pred_gt_equal = 0
  tot_pix = 0
  # print(output.shape)
  # print(label.shape)
  for i in range(output.shape[2]):
    for j in range(output.shape[3]):
      if val == True and label[0][i][j] == 0:
        continue
      tot_pix += 1
      if output[0][0][i][j] > output[0][1][i][j] and output[0][0][i][j] > output[0][2][i][j]:
        pred = 0
      if output[0][1][i][j] > output[0][0][i][j] and output[0][1][i][j] > output[0][2][i][j]:
        pred = 1
      if output[0][2][i][j] > output[0][1][i][j] and output[0][2][i][j] > output[0][0][i][j]:
        pred = 2
      
      #Confusion Matrix
      # print(label[])
      if val == False:
        confu_mat[label[0][i][j]][pred] += 1 
        if pred == label[0][i][j]:
          pred_gt_equal += 1
      else:
        confu_mat[label[0][i][j] - 1][pred] += 1 
        if pred == label[0][i][j] - 1:
          pred_gt_equal += 1
  return confu_mat


def cal_dice_coefficient(output, label, val):
  output = output.cpu().detach()
  output = np.array(output)
  label = label.cpu().detach()
  label = np.array(label)
  confu_mat = np.zeros((3, 3))
  pred_gt_equal = 0
  tot_pix = 0
  # print(output.shape)
  # print(label.shape)
  for i in range(output.shape[2]):
    for j in range(output.shape[3]):
      if val == True and label[0][i][j] == 0:
        continue
      # print('here')
      tot_pix += 1
      if output[0][0][i][j] > output[0][1][i][j] and output[0][0][i][j] > output[0][2][i][j]:
        pred = 0
      if output[0][1][i][j] > output[0][0][i][j] and output[0][1][i][j] > output[0][2][i][j]:
        pred = 1
      if output[0][2][i][j] > output[0][1][i][j] and output[0][2][i][j] > output[0][0][i][j]:
        pred = 2
      if val == False:
        if pred == label[0][i][j]:
          pred_gt_equal += 1
      else:
        
        if pred == label[0][i][j] - 1:
          pred_gt_equal += 1
  if tot_pix == 0:
    return -1
  dice_coefficient = pred_gt_equal/ (2 * tot_pix)
  return dice_coefficient  

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
learning_rate = 0.0001
epochs = 10
model = SegNet()
if torch.cuda.is_available():
  model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate, momentum = 0.9, weight_decay = 0.005)
        

In [ ]:
if torch.cuda.is_available():
  print('true')
else:
  print('false')

In [ ]:
import time
min_loss = 1
for i in range(epochs):
        start_time = time.time()
        mean_loss = 0
        tot = 0
        idx = 0
        confusion_matrix = np.zeros((3, 3))
        dice_coefficient = 0
        for img, label in my_dataloader:
            
            img_r = torch.unsqueeze(img, 1)
            # print(img_r.shape)
            if torch.cuda.is_available():
              img_r = img_r.cuda()
            # label_r = torch.unsqueeze(label, 1)
            label = label.type(torch.LongTensor)
            if torch.cuda.is_available():
              label = label.cuda()
            optimizer.zero_grad()
            # print(img_r.shape)
            output, output_s = model(img_r)
            # print(output.shape, label.shape)
            loss = criterion(output, label)
            # loss2 = dice_loss(label, output)
            # print(loss2.item())
            confusion_matrix += cal_confusion_matrix(output_s, label, False)
            dice_coefficient += cal_dice_coefficient(output_s, label, False)
            # tot += loss2.item()
            loss.backward()
            optimizer.step()
            mean_loss += loss
            idx += 1

        print(idx)  
        mean_loss /= idx
        tot /= idx
        dice_coefficient /= idx
        end_time = time.time()
        elapse_time = end_time - start_time
        print(f'epoch {i} loss: {mean_loss}, elapse time: {elapse_time}',  dice_coefficient)
        for i in range(confusion_matrix.shape[0]):
          for j in range(confusion_matrix.shape[1]):
            print(confusion_matrix[i][j], end = " ")
          print()




        
        torch.save(model, 'segnet_model_deconv_conv' + str(epochs))
            
            # state_dict = model.module.state_dict()
            # for key in state_dict.keys():
            #     state_dict[key] = state_dict[key].cpu()
            #     torch.save(state_dict, 'segnet_weight_baseline.pth')

In [ ]:
output_s[0][2][1][1]

In [ ]:
import time
# model = torch.load('segnet_model_')
# model.eval()
mean_loss_v = 0
tot_v = 0
idx = 0
tot_correct = 0
tot_pixel = 0
dice_coefficient = 0
confusion_matrix_v = np.zeros((3, 3))
with torch.no_grad():
  for img_v, label_v in my_valoader:
      
      img_r = torch.unsqueeze(img_v, 1)
      if torch.cuda.is_available():
        img_r = img_r.cuda()
      # img = img_v.type(torch.FloatTensor)
      # label_r = torch.unsqueeze(label_v, 1)
      label_v = label_v.type(torch.LongTensor)
      if torch.cuda.is_available():
        label_v = label_v.cuda()
      output, output_s = model(img_r)
      # print(output.shape, label.shape)
      # loss = criterion(output_s, label_v)
      # loss2 = dice_loss(label, output)
      # print(loss2.item())
      # print(type(output))
      confusion_matrix_v += cal_confusion_matrix(output_s, label_v, True)
      # print(cal_confusion_matrix(output_s, label_v, True))
      if cal_dice_coefficient(output_s, label_v, True) != -1:
        dice_coefficient += cal_dice_coefficient(output_s, label_v, True)
      else:
        idx -= 1
      # print(mat)
      
      # print(confusion_matrix_v)
      
      # tot_v += loss2.item()
      
      
      # mean_loss_v += loss
      idx += 1

# print(idx_v)  
# mean_loss_v /= idx
tot_v /= idx
dice_coefficient /= idx
end_time = time.time()
# elapse_time = end_time - start_time
# print(f'epoch {i} elapse time: {elapse_time}', dice_coefficient)
print(dice_coefficient)
print(confusion_matrix_v)

In [ ]:

model = torch.load('segnet_model_deconv_conv10')
model.eval()
from torchsummary import summary
summary(model, input_size = (1, 32, 32), batch_size = 1, device = 'cuda')